This notebook contains the same model as that present in the Capstone Project.ipynb.
The only difference that lies here is that I have used tensorflow.summary module in order to store the values of all the tensors that are present and have been used throughout the model, so that I can get visualizations for the project model using tensorflow's tensorboard module.
For the documetation please refer to the Capstone Project.ipynb file. The codes are 100% the same except for the usage of tf.summary.

In [1]:
import pandas as pd
import os
DATA_DIR = "E:\\train"
names=[]
i=0
for root, dirs, files in os.walk(DATA_DIR,topdown=False):
    for name in files:
        names.append(name.replace('.jpeg',''))
print(len(names))
df_Org=pd.read_csv("E:/trainLabels.csv/trainLabels.csv")
sf=df_Org.values.tolist()

df=[]
for name in names:
    for i in range (0,len(sf)):
        if(sf[i][0]==name):
            df.append(sf[i])
print(len(df))

1427
1427


In [2]:
import cv2
import numpy as np
x=[]
y=[]
for i in range (0,len(df)-1):
    img = cv2.imread("E://train//"+df[i][0]+".jpeg")
    img = cv2.resize(img,(512,340))
    img = np.array(img).reshape((340,512,3))
    x.append(img)
    y.append(df[i][1])
    i+=1
x=np.array(x)
y=np.array(y)
y=np.array(y).reshape(len(y),1)
print(x.shape)
print(y.shape)

(1426, 340, 512, 3)
(1426, 1)


In [3]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2987, random_state=42)
X_train=np.split(x_train,10)
y_train=np.split(y_train,10)
print(np.array(X_train).shape)
print(np.array(y_train).shape)

X_test=x_test[:400]
Y_test=y_test[:400]
x_test=np.split(X_test,4)
y_test=np.split(Y_test,4)

def one_hot_encode(x):
    from sklearn.preprocessing import OneHotEncoder
    enc=OneHotEncoder(n_values=5)
    ohc_labels=enc.fit_transform(np.array(x).reshape(-1,1)).toarray()
    return ohc_labels

(10, 100, 340, 512, 3)
(10, 100, 1)


In [4]:
import tensorflow as tf

with tf.name_scope('input'):
    x = tf.placeholder("float", shape=[None,340,512,3], name='x')
    y = tf.placeholder("float", shape=[None,5], name='y')

with tf.name_scope('dropout'):
    keep_probs = tf.placeholder(tf.float32, name='keep_probs')
    tf.summary.scalar('dropout_keep_probability', keep_probs)

c:\users\hp\anaconda3\envs\testenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def conv2d(inp_lyr,ofilters,kernel,stride):
    conv2d_lyr=tf.layers.conv2d(inputs=inp_lyr,
                               filters=ofilters,
                               kernel_size=kernel,
                               strides=stride,
                               padding="same",
                               activation=tf.nn.relu,
                               )
    return conv2d_lyr

def maxpool2d(inp_lyr,pool,stride):
    maxp2d=tf.layers.max_pooling2d(inputs=inp_lyr,
                                 pool_size=pool,
                                 strides=stride,
                                 padding="same")
    return maxp2d

In [6]:
def model():
    img_inp=tf.reshape(x,[-1,340,512,3])
    layer=conv2d(img_inp,32,[7,7],[2,2])
    layer=maxpool2d(layer,[5,5],[2,2])
    
    layer=conv2d(layer,32,[5,5],[1,1])
    layer=conv2d(layer,64,[3,3],[1,1])
    layer=maxpool2d(layer,[3,3],[2,2])
    
    layer=conv2d(layer,64,[3,3],[1,1])
    layer=conv2d(layer,128,[3,3],[1,1])
    layer=conv2d(layer,256,[3,3],[1,1])
    layer=maxpool2d(layer,[3,3],[2,2])
    
    layer=tf.layers.dropout(layer,0.4)
    
    layer=tf.layers.flatten(layer)
    
    layer=tf.layers.dense(layer,256)
    layer=tf.layers.dense(layer,512)
    layer=tf.layers.dense(layer,1024)
    
    layer=tf.layers.dense(layer,5)
    
    layer=tf.nn.softmax(layer)
    
    return layer

In [7]:
def train_model():
    for epoch in range(15):
        for i in range (0,10):
            x_img=X_train[i]
            y_img=one_hot_encode(y_train[i])
            train_summary, train_acc = sess.run([merged, acc], feed_dict={x:x_img,
                                                                                y: y_img,
                                                                                keep_probs: 1.0})
            train_writer.add_summary(train_summary, epoch+1+(i+1)/100)
            print('Epoch: {} Batch: {} Accuracy = {:.6f}'.format(epoch+1, i+1, train_acc))
            sess.run(epoch_batch,feed_dict={x: x_img, y: y_img, keep_probs: 0.5})

In [8]:
cnn_mod=model()
cnn_mod=tf.identity(cnn_mod, name='model')
with tf.name_scope('cross_entropy'):
    cross_entropy=-tf.reduce_sum(y*tf.log(cnn_mod))
tf.summary.scalar('cross_entropy',cross_entropy)

with tf.name_scope('training'):
    epoch_batch=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_preds'):
        corr_pred=tf.equal(tf.argmax(cnn_mod, 1), tf.argmax(y, 1))
        
    with tf.name_scope('accuracy'):
        acc=tf.reduce_mean(tf.cast(corr_pred, tf.float32), name='acc')
tf.summary.scalar('acc',acc)

<tf.Tensor 'acc:0' shape=() dtype=string>

In [9]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())  
merged=tf.summary.merge_all()
train_writer = tf.summary.FileWriter('F:/Work/Summaries' + '/train', sess.graph)
train_model()

Epoch: 1 Batch: 1 Accuracy = 0.050000
Epoch: 1 Batch: 2 Accuracy = 0.740000
Epoch: 1 Batch: 3 Accuracy = 0.700000
Epoch: 1 Batch: 4 Accuracy = 0.760000
Epoch: 1 Batch: 5 Accuracy = 0.720000
Epoch: 1 Batch: 6 Accuracy = 0.690000
Epoch: 1 Batch: 7 Accuracy = 0.670000
Epoch: 1 Batch: 8 Accuracy = 0.770000
Epoch: 1 Batch: 9 Accuracy = 0.710000
Epoch: 1 Batch: 10 Accuracy = 0.770000
Epoch: 2 Batch: 1 Accuracy = 0.710000
Epoch: 2 Batch: 2 Accuracy = 0.740000
Epoch: 2 Batch: 3 Accuracy = 0.700000
Epoch: 2 Batch: 4 Accuracy = 0.760000
Epoch: 2 Batch: 5 Accuracy = 0.720000
Epoch: 2 Batch: 6 Accuracy = 0.690000
Epoch: 2 Batch: 7 Accuracy = 0.670000
Epoch: 2 Batch: 8 Accuracy = 0.770000
Epoch: 2 Batch: 9 Accuracy = 0.710000
Epoch: 2 Batch: 10 Accuracy = 0.770000
Epoch: 3 Batch: 1 Accuracy = 0.710000
Epoch: 3 Batch: 2 Accuracy = 0.740000
Epoch: 3 Batch: 3 Accuracy = 0.700000
Epoch: 3 Batch: 4 Accuracy = 0.760000
Epoch: 3 Batch: 5 Accuracy = 0.720000
Epoch: 3 Batch: 6 Accuracy = 0.690000
Epoch: 3 B

In [10]:
save_model_path='F:/Work/Summaries/capstone'
train_writer.close()
saver = tf.train.Saver()
save_path = saver.save(sess, save_model_path)